In [1]:
%cd ..

/home/romet/projects/ut/e2e-rally-estonia


In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torchvision

from dataloading.nvidia import NvidiaDataset, NvidiaValidationDataset, Normalize
from pilotnet import PilotnetControl, PilotNetConditional
from metrics.metrics import calculate_open_loop_metrics
from viz.analytics import create_waypoint_error_plot

from metrics.metrics import calculate_closed_loop_metrics, calculate_trajectory_open_loop_metrics, calculate_open_loop_metrics, read_frames_driving, read_frames_expert
from trajectory import calculate_steering_angle
import math

from trainer import ControlTrainer, ConditionalTrainer
from torchvision import transforms

from camera_frame import CameraFrameTransformer


%load_ext autoreload 
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def load_dataset(paths):
        return NvidiaDataset(paths,
                             transform=tr,
                             output_modality="steering_angle", 
                             metadata_file="nvidia_frames.csv")

In [4]:
tr = transforms.Compose([Normalize()])

data_path = Path("/home/romet/data/datasets/rally-estonia/dataset-romet-thesis")
data2_path = Path("/home/romet/data2/datasets/rally-estonia/dataset-romet-thesis")
dataset_path = Path("/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021")


steering_1_forward = load_dataset([data_path / "2022-06-29-15-09-08_e2e_elva__forward_steering1"])
steering_1_backward = load_dataset([data2_path / "2022-06-29-15-45-24_e2e_elva__backward_steering1-2"])

steering_2_forward = load_dataset([data_path / "2022-06-29-10-46-40_e2e_elva__forward_steering2"])
steering_2_backward = load_dataset([data_path / "2022-06-29-11-21-56_e2e_elva__backward_steering2"])

steering_overfit_forward = load_dataset([data_path / "2022-06-29-16-26-28_e2e_elva__forward_steering_overfit"])
steering_overfit_backward = load_dataset([data_path / "2022-06-29-17-02-55_e2e_elva_backward_steering_overfit"])

waypoints_1_forward = load_dataset([data2_path / "2022-06-28-16-54-49_e2e_elva__forward_waypoints_bal"])
waypoints_1_backward = load_dataset([data2_path / "2022-06-28-17-29-21_e2e_elva__backwards_waypoints_bal"])

waypoints_2_forward = load_dataset([data_path / "2022-06-29-12-18-31_e2e_elva__forward_trajectory_2"])
waypoints_2_backward = load_dataset([data_path / "2022-06-29-12-57-41_e2e_elva__backward_trajectory_2"])

waypoints_overfit_forward = load_dataset([data_path / "2022-06-29-18-05-32_e2e_elva_forward_waypoinit_overfit",
                                   data_path / "2022-06-29-18-21-39_e2e_elva_forward_waypoinit_overfit2"])
waypoints_overfit_backward = load_dataset([data_path / "2022-06-29-18-42-10_e2e_elva_backward_waypoint_overfit"])

expert_dataset = load_dataset([dataset_path / "2021-10-26-10-49-06_e2e_rec_ss20_elva"])

/home/romet/data/datasets/rally-estonia/dataset-romet-thesis/2022-06-29-15-09-08_e2e_elva__forward_steering1: lenght=61296, filtered=13
/home/romet/data2/datasets/rally-estonia/dataset-romet-thesis/2022-06-29-15-45-24_e2e_elva__backward_steering1-2: lenght=65355, filtered=1
/home/romet/data/datasets/rally-estonia/dataset-romet-thesis/2022-06-29-10-46-40_e2e_elva__forward_steering2: lenght=62086, filtered=16
/home/romet/data/datasets/rally-estonia/dataset-romet-thesis/2022-06-29-11-21-56_e2e_elva__backward_steering2: lenght=66483, filtered=9
/home/romet/data/datasets/rally-estonia/dataset-romet-thesis/2022-06-29-16-26-28_e2e_elva__forward_steering_overfit: lenght=60080, filtered=17
/home/romet/data/datasets/rally-estonia/dataset-romet-thesis/2022-06-29-17-02-55_e2e_elva_backward_steering_overfit: lenght=65479, filtered=1
/home/romet/data2/datasets/rally-estonia/dataset-romet-thesis/2022-06-28-16-54-49_e2e_elva__forward_waypoints_bal: lenght=61207, filtered=21
/home/romet/data2/datasets/

In [5]:
expert_root_path = Path("/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021")
expert_ds = [expert_root_path / dataset_path for dataset_path in 
             ["2022-06-10-13-23-01_e2e_elva_forward", "2022-06-10-13-03-20_e2e_elva_backward"]]
             #['2021-10-26-10-49-06_e2e_rec_ss20_elva', '2021-10-26-11-08-59_e2e_rec_ss20_elva_back']]
expert_ds = [
    expert_root_path / '2021-10-26-10-49-06_e2e_rec_ss20_elva',
    expert_root_path / '2021-10-26-11-08-59_e2e_rec_ss20_elva_back'
]
expert_frames = read_frames_expert(expert_ds, "nvidia_frames.csv")


drive_ds = [
    data_path / "2022-06-29-15-09-08_e2e_elva__forward_steering1",
    data2_path / "2022-06-29-15-45-24_e2e_elva__backward_steering1-2"
]

drives = {
    "Steering 1": [
        data_path / "2022-06-29-15-09-08_e2e_elva__forward_steering1", 
        data2_path / "2022-06-29-15-45-24_e2e_elva__backward_steering1-2"
    ],
    "Steering 2": [
        data_path / "2022-06-29-10-46-40_e2e_elva__forward_steering2",
        data_path / "2022-06-29-11-21-56_e2e_elva__backward_steering2"
    ],
    "Steering overfit": [
        data_path / "2022-06-29-16-26-28_e2e_elva__forward_steering_overfit",
        data_path / "2022-06-29-17-02-55_e2e_elva_backward_steering_overfit"
    ],
    "Steering total": [
        data_path / "2022-06-29-15-09-08_e2e_elva__forward_steering1", 
        data2_path / "2022-06-29-15-45-24_e2e_elva__backward_steering1-2",
        data_path / "2022-06-29-10-46-40_e2e_elva__forward_steering2",
        data_path / "2022-06-29-11-21-56_e2e_elva__backward_steering2",
        data_path / "2022-06-29-16-26-28_e2e_elva__forward_steering_overfit",
        data_path / "2022-06-29-17-02-55_e2e_elva_backward_steering_overfit"
    ],
    
    
    
    "Waypoints 1": [
        data2_path / "2022-06-28-16-54-49_e2e_elva__forward_waypoints_bal", 
        data2_path / "2022-06-28-17-29-21_e2e_elva__backwards_waypoints_bal"
    ],
    "Waypoints 2": [
        data_path / "2022-06-29-12-18-31_e2e_elva__forward_trajectory_2",
        data_path / "2022-06-29-12-57-41_e2e_elva__backward_trajectory_2"
    ],
    "Waypoints overfit": [
        data_path / "2022-06-29-18-05-32_e2e_elva_forward_waypoinit_overfit",
        data_path / "2022-06-29-18-21-39_e2e_elva_forward_waypoinit_overfit2",
        data_path / "2022-06-29-18-42-10_e2e_elva_backward_waypoint_overfit"
    ],
    "Waypoints total": [
        data2_path / "2022-06-28-16-54-49_e2e_elva__forward_waypoints_bal", 
        data2_path / "2022-06-28-17-29-21_e2e_elva__backwards_waypoints_bal",
        data_path / "2022-06-29-12-18-31_e2e_elva__forward_trajectory_2",
        data_path / "2022-06-29-12-57-41_e2e_elva__backward_trajectory_2",
        data_path / "2022-06-29-18-05-32_e2e_elva_forward_waypoinit_overfit",
        data_path / "2022-06-29-18-21-39_e2e_elva_forward_waypoinit_overfit2",
        data_path / "2022-06-29-18-42-10_e2e_elva_backward_waypoint_overfit"
    ]
}

metrics_df = pd.DataFrame(columns=["model", 'mae', 'rmse', 'max', 'failure_rate', 
                                   'total_distance', 'autonomous_distance', 'autonomous_pct',
                                   'interventions', 'distance_per_intervention', 
                                   'whiteness', 'cmd_whiteness', 'expert_whiteness'])
for model, drive_ds in drives.items():
    print(f"Calculating metrics for {model}")
    drive_frames = read_frames_driving(drive_ds, "nvidia_frames.csv")
    drive_metrics = calculate_closed_loop_metrics(drive_frames, expert_frames, fps=30)
    metrics = pd.DataFrame(data=drive_metrics, index=[0])
    metrics['model'] = model
    metrics_df = metrics_df.append(metrics, ignore_index=True)
    
metrics_df.set_index("model");
#metrics_df = metrics_df[["model", 'mae', 'rmse', 'max', 'failure_rate', 'distance', 'distance_per_intervention', 'interventions', 'whiteness', 'cmd_whiteness', 'expert_whiteness']]
metrics_df
    

Calculating metrics for Steering 1
Calculating metrics for Steering 2
Calculating metrics for Steering overfit
Calculating metrics for Steering total
Calculating metrics for Waypoints 1
Calculating metrics for Waypoints 2
Calculating metrics for Waypoints overfit
Calculating metrics for Waypoints total


,model,mae,rmse,max,failure_rate,total_distance,autonomous_distance,autonomous_pct,interventions,distance_per_intervention,whiteness,cmd_whiteness,expert_whiteness
0,Steering 1,0.319864,0.435649,1.719248,3.976351,20290.074234,19684.56301,0.970157,27,729.057889,33.048776,64.228219,23.387318
1,Steering 2,0.322203,0.437125,1.72663,3.546813,20314.145752,19847.528468,0.97703,28,708.840302,31.940696,73.352538,23.387318
2,Steering overfit,0.324937,0.431126,1.723538,2.957432,20295.150579,19990.267549,0.984978,14,1427.876254,32.041797,50.980124,23.387318
3,Steering total,0.322346,0.434631,1.72663,3.490936,60922.584213,59545.572674,0.977397,69,862.979314,32.502578,63.583308,23.387318
4,Waypoints 1,0.321549,0.42801,1.724372,3.173196,20415.125827,20177.924789,0.988381,11,1834.356799,39.121676,183.843477,23.387318
5,Waypoints 2,0.35998,0.475967,1.724342,4.506464,20304.899821,19962.134788,0.983119,18,1109.007488,44.269527,125.349357,23.387318
6,Waypoints overfit,0.360102,0.484031,1.721164,5.570047,20267.001525,20004.343247,0.98704,13,1538.795634,36.200157,105.48292,23.387318
7,Waypoints total,0.34705,0.463108,1.724372,4.408172,61014.802664,60172.178315,0.98619,43,1399.352984,40.039572,142.465557,23.387318


In [6]:
metrics_df

,model,mae,rmse,max,failure_rate,total_distance,autonomous_distance,autonomous_pct,interventions,distance_per_intervention,whiteness,cmd_whiteness,expert_whiteness
0,Steering 1,0.319864,0.435649,1.719248,3.976351,20290.074234,19684.56301,0.970157,27,729.057889,33.048776,64.228219,23.387318
1,Steering 2,0.322203,0.437125,1.72663,3.546813,20314.145752,19847.528468,0.97703,28,708.840302,31.940696,73.352538,23.387318
2,Steering overfit,0.324937,0.431126,1.723538,2.957432,20295.150579,19990.267549,0.984978,14,1427.876254,32.041797,50.980124,23.387318
3,Steering total,0.322346,0.434631,1.72663,3.490936,60922.584213,59545.572674,0.977397,69,862.979314,32.502578,63.583308,23.387318
4,Waypoints 1,0.321549,0.42801,1.724372,3.173196,20415.125827,20177.924789,0.988381,11,1834.356799,39.121676,183.843477,23.387318
5,Waypoints 2,0.35998,0.475967,1.724342,4.506464,20304.899821,19962.134788,0.983119,18,1109.007488,44.269527,125.349357,23.387318
6,Waypoints overfit,0.360102,0.484031,1.721164,5.570047,20267.001525,20004.343247,0.98704,13,1538.795634,36.200157,105.48292,23.387318
7,Waypoints total,0.34705,0.463108,1.724372,4.408172,61014.802664,60172.178315,0.98619,43,1399.352984,40.039572,142.465557,23.387318


In [53]:
metrics_df["interventions"] = [18, 12, 10, 40, 6, 10, 8, 24]
metrics_df["crossroads"] = [6/11, 9/11, 9/11, 24/33, 10/11, 8/11, 9/11, 27/33]
metrics_df["crossroads"] = metrics_df["crossroads"] * 100
metrics_df["autonomous_pct"] = metrics_df["autonomous_pct"] * 100
metrics_df["distance_per_intervention"] = metrics_df.apply(lambda x: x['total_distance']/x['interventions'] if x['interventions'] != 0 else x['total_distance'], axis=1)
metrics_df

,model,total_distance,autonomous_pct,interventions,distance_per_intervention,crossroads,cl_mae,cl_whiteness,cl_cmd_whiteness,failure_rate
0,Steering 1,20290.074234,9701572691.322632,18,1127.226346,54.545455,0.319864,33.048776,64.228219,3.976351
1,Steering 2,20314.145752,9770299332.22613,12,1692.845479,81.818182,0.322203,31.940696,73.352538,3.546813
2,Steering overfit,20295.150579,9849775428.685524,10,2029.515058,81.818182,0.324937,32.041797,50.980124,2.957432
3,Steering total,60922.584213,9773973550.785933,40,1523.064605,72.727273,0.322346,32.502578,63.583308,3.490936
4,Waypoints 1,20415.125827,9883811131.24815,6,3402.520971,90.909091,0.321549,39.121676,183.843477,3.173196
5,Waypoints 2,20304.899821,9831190975.673828,10,2030.489982,72.727273,0.35998,44.269527,125.349357,4.506464
6,Waypoints overfit,20267.001525,9870401017.090096,8,2533.375191,81.818182,0.360102,36.200157,105.48292,5.570047
7,Waypoints total,61014.802664,9861898373.519016,24,2542.283444,81.818182,0.34705,40.039572,142.465557,4.408172


In [8]:
metrics_df.rename(columns={'whiteness': 'cl_whiteness'}, inplace=True)
metrics_df.rename(columns={'cmd_whiteness': 'cl_cmd_whiteness'}, inplace=True)
metrics_df.rename(columns={'mae': 'cl_mae'}, inplace=True)

In [9]:
metrics_df.drop(['rmse', 'max', 'expert_whiteness'], axis=1, inplace=True)

In [10]:
cols = metrics_df.columns.tolist()
print(cols)
order=['model', 'total_distance', 'autonomous_pct', 'interventions','distance_per_intervention', 'crossroads',
       'cl_mae', 'cl_whiteness', 'cl_cmd_whiteness', 'failure_rate',
       #'ol_mae','ol_season_whiteness'
      ]
metrics_df=metrics_df[order]

['model', 'cl_mae', 'failure_rate', 'total_distance', 'autonomous_distance', 'autonomous_pct', 'interventions', 'distance_per_intervention', 'cl_whiteness', 'cl_cmd_whiteness', 'crossroads']


In [13]:
metrics_df["autonomous_pct"] = metrics_df["autonomous_pct"] * 100

In [13]:
metrics_df.style.format({
    'cl_mae': "{:.4f}m",
    'rmse': "{:.4f}m",
    'max': "{:.4f}m",
    'failure_rate': "{:.2f}%",
    'interventions': "{:.0f}",
    'cl_whiteness': "{:.2f}°/s",
    'ol_whiteness': "{:.2f}°/s",
    'cl_cmd_whiteness': "{:.2f}°/s",
    'ol_season_whiteness': "{:.2f}°/s",
    'ol_mae': "{:.2f}°",
    'total_distance': "{:.0f}m",
    'autonomous_pct': "{:.2f}%",
    'distance_per_intervention': "{:.0f}m",
    'crossroads': "{:.2f}%",
})

,model,total_distance,autonomous_pct,interventions,distance_per_intervention,crossroads,cl_mae,cl_whiteness,cl_cmd_whiteness,failure_rate
0,Steering 1,20290m,9701.57%,18,1127m,54.55%,0.3199m,33.05°/s,64.23°/s,3.98%
1,Steering 2,20314m,9770.30%,12,1693m,81.82%,0.3222m,31.94°/s,73.35°/s,3.55%
2,Steering overfit,20295m,9849.78%,10,2030m,81.82%,0.3249m,32.04°/s,50.98°/s,2.96%
3,Steering total,60923m,9773.97%,38,1603m,72.73%,0.3223m,32.50°/s,63.58°/s,3.49%
4,Waypoints 1,20415m,9883.81%,6,3403m,90.91%,0.3215m,39.12°/s,183.84°/s,3.17%
5,Waypoints 2,20305m,9831.19%,10,2030m,72.73%,0.3600m,44.27°/s,125.35°/s,4.51%
6,Waypoints overfit,20267m,9870.40%,8,2533m,81.82%,0.3601m,36.20°/s,105.48°/s,5.57%
7,Waypoints total,61015m,9861.90%,24,2542m,81.82%,0.3471m,40.04°/s,142.47°/s,4.41%


In [27]:
def load_model(model_name):
    model = PilotNetConditional(n_branches=3)
    model.load_state_dict(torch.load(f"models/{model_name}/best.pt"))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    return model

In [41]:
from trainer import ConditionalTrainer

def calculate_metrics(model, dataset):
    dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=16, pin_memory=True,
                            persistent_workers=True)

    trainer = ConditionalTrainer(n_conditional_branches=3)
    steering_predictions = trainer.predict(model, dataloader)
    true_steering_angles = dataloader.dataset.frames.steering_angle.to_numpy()
    metrics = calculate_open_loop_metrics(steering_predictions, true_steering_angles, fps=30)
    return metrics

In [44]:
tr = transforms.Compose([Normalize()])
#valid_dataset = NvidiaDataset([
#    expert_root_path / "2022-06-10-13-23-01_e2e_elva_forward",
#    expert_root_path / "2022-06-10-13-03-20_e2e_elva_backward"
#], n_branches=3)

valid_dataset = NvidiaValidationDataset(root_path=expert_root_path)

/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-05-28-15-19-48_e2e_sulaoja_20_30: lenght=10705, filtered=3
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-20-07_e2e_rec_ss6: lenght=25833, filtered=4
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-06-31_e2e_rec_ss6: lenght=3002, filtered=1
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-09-18_e2e_rec_ss6: lenght=4550, filtered=2
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-36-16_e2e_rec_ss6: lenght=25351, filtered=18
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-09-24-14-03-45_e2e_rec_ss11_backwards: lenght=25167, filtered=5
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-10-26-10-49-06_e2e_rec_ss20_elva: lenght=33034, filtered=11
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-10-26-

In [49]:
steering_1_model = load_model("20220627193827_steering-conditional-pre")
calculate_metrics(steering_1_model, valid_dataset)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [45]:
steering_2_model = load_model("20220628234819_steering-conditional-pre-2")
calculate_metrics(steering_2_model, valid_dataset)

  0%|          | 0/4411 [00:00<?, ?it/s]

{'mae': 7.396755266897202,
 'rmse': 19.89840094590981,
 'max': 407.48724354540315,
 'whiteness': 83.60266,
 'expert_whiteness': 25.28222219921574}

In [47]:
steering_overfit_model = load_model("20220629083303_steering-conditional-pre-overfit")
calculate_metrics(steering_overfit_model, valid_dataset)

  0%|          | 0/4411 [00:00<?, ?it/s]

{'mae': 6.3415908293058765,
 'rmse': 14.96046022016641,
 'max': 332.1224589264579,
 'whiteness': 68.92555,
 'expert_whiteness': 25.28222219921574}